## Project Description: Next Word Prediction Using LSTM
#### Project Overview:

This project aims to develop a deep learning model for predicting the next word in a given sequence of words. The model is built using Long Short-Term Memory (LSTM) networks, which are well-suited for sequence prediction tasks. The project includes the following steps:

1- Data Collection: We use the text of Shakespeare's "Hamlet" as our dataset. This rich, complex text provides a good challenge for our model.

2- Data Preprocessing: The text data is tokenized, converted into sequences, and padded to ensure uniform input lengths. The sequences are then split into training and testing sets.

3- Model Building: An LSTM model is constructed with an embedding layer, two LSTM layers, and a dense output layer with a softmax activation function to predict the probability of the next word.

4- Model Training: The model is trained using the prepared sequences, with early stopping implemented to prevent overfitting. Early stopping monitors the validation loss and stops training when the loss stops improving.

5- Model Evaluation: The model is evaluated using a set of example sentences to test its ability to predict the next word accurately.

6- Deployment: A Streamlit web application is developed to allow users to input a sequence of words and get the predicted next word in real-time.

In [ ]:
## Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

## Load the data
data = gutenberg.raw('shakespeare-hamlet.txt')
## save to a file
with open('hamlet.txt', 'w') as f:
    f.write(data)

In [ ]:
## Data Preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## Load the data
with open('hamlet.txt', 'r') as file:
    data = file.read().lower()

## Tokenization creating idex for words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
total_words = len(tokenizer.word_index)+1


In [ ]:
## Create imput sequences
input_sequences = []

for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
## Padding sequences
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))


In [ ]:
## Create predictors and labels
import tensorflow as tf
predictors, Label = input_sequences[:,:-1], input_sequences[:,-1]
Label = tf.keras.utils.to_categorical(Label, num_classes=total_words)

In [ ]:
## Split the data into training and testing
x_train, x_test, y_train, y_test = train_test_split(predictors, Label, test_size=0.2, random_state=42)

In [ ]:
## Model Building
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

## Model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
## Early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15)

In [ ]:
## Model Training
model.fit(x_train, y_train, epochs=100, validation_data = (x_test,y_test), verbose=1)

In [ ]:
## Save the model
model.save('nect_word_lstm.keras')
## Save the tokenizer
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f,pickle.HIGHEST_PROTOCOL)

In [ ]:
## function to predict the next word
def predict_next_word(model,tokenizer,text,max_sequence_length):
        token_list = tokenizer.texts_to_sequences([text])[0]
        if len(token_list) > max_sequence_length:
            token_list = token_list[-max_sequence_length:]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=1)
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                return word
        return None

In [ ]:
input_text = "To be or not to be"
print(f"Input Text: {input_text}")
max_sequence_length = model.input_shape[1]+1
predicted_word = predict_next_word(model, tokenizer, input_text, max_sequence_length)
print(f"Predicted Word: {predicted_word}")